# Understanding GPT: How To Implement a Simple GPT Model with PyTorch
This guide provides a step-by-step explanation of implementing a simple GPT (Generative Pre-trained Transformer) model using PyTorch. We'll cover the necessary components, how to train the model, and how to generate text. 



## Table of Contents

- [Introduction](#introduction)
- [Key Terms](#key-terms)
- [Setup](#setup)
- [Creating the Dataset](#creating-the-dataset)
- [Building the GPT Model](#building-the-gpt-model)
- [Training the Model](#training-the-model)
- [Generating Text](#generating-text)
- [Conclusion](#conclusion)

## Introduction
The GPT model is a transformer-based model designed for natural language processing tasks, such as text generation. This tutorial will guide you through creating a simplified version of GPT, training it on a small dataset, and generating text.


## Key Terms
- **Transformer**: A model architecture primarily used for natural language processing tasks. It uses self-attention mechanisms to process input sequences.
- **Attention Mechanism**: A technique that allows the model to focus on different parts of the input sequence when generating the output.
- **Embedding**: A representation of input data (e.g., words) as vectors in a continuous vector space.
- **LayerNorm**: A normalization technique applied to the output of each layer in a neural network to stabilize and accelerate training.
- **MultiheadAttention**: A mechanism that applies multiple attention functions in parallel to capture different aspects of the input sequence.

## Setup

First, ensure you have the required libraries installed. You can install them using pip:



In [ ]:
pip install torch transformers

## Creating the Dataset

We start by creating a custom dataset class to handle text inputs and tokenization. The ```SimpleDataset``` class uses the GPT-2 tokenizer to encode the text data

### Explanation:

This code is creating a simple dataset using the [PyTorch](https://pytorch.org/) framework that will be used to train or test a machine learning model. 

[PyTorch's](https://pytorch.org/) **Dataset** and **DataLoader** classes are used in this code as a way to organize data in a format that PyTorch can use effectively. This  example is utilizing a specific library called "Transformers" (by [Hugging Face](https://huggingface.co/)) that contains pre-built utilities for text tokenization and Transformer models like GPT-2.

1. **GPT2Tokenizer** class is used to tokenize the given text. Tokenization is essentially breaking down text into smaller chunks (tokens), which is a crucial step in many natural language processing tasks. This tokenizer also converts these tokens into their corresponding IDs that the model can understand.

2. **SimpleDataset** class is defined inheriting from [PyTorch's](https://pytorch.org/) **Dataset** class:

    - **\__init\__** method initializes the object. It is using three parameters: texts, tokenizer, and max_length.
    
    - **\__len\__** method returns the length of the dataset. In this case, it is the total number of texts to be processed.

    - **\__getitem\__** method is used to get an item at a particular index idx. It applies the tokenizer to the text and generates an "encoding," which involves transforming the human-readable text into a machine-readable format. The encoding also includes attention masks which is a binary mask indicating the position of the padded indices so that the model does not attend to them.

3. Once the **SimpleDataset** object is initialized, a [PyTorch](https://pytorch.org/) **DataLoader** is created using the dataset, with a specified batch size of 2 and shuffling enabled. The DataLoader class provides an iterator that serves batches of the dataset, which is highly useful in training models in batches.

**Shuffling** is the process of randomizing the order of data samples in a dataset. It is an important step because it ensures that each data point helps in the learning process independently of other data points. This provides two main benefits:
1. **Prevents overfitting**: If the model sees similar samples one after the other (like when the data is sorted), it may overfit to those data points and perform poorly on unseen data. Shuffling can help to break up these ordered datasets and ensure a good mix of samples from different classes in the training data.
2. **Improvement in the model's performance**: Data shuffling helps the model's optimization algorithm (such as Stochastic Gradient Descent) to achieve better performance, as it introduces randomness into the optimization process, which can help the model to escape from local minima in the optimization landscape.

Here's an example of shuffling in Python:
```Python
import random

data = [1, 2, 3, 4, 5]
random.shuffle(data)
print(data)

# Output might be: [3, 4, 2, 5, 1]
# The output is randomized every time this is run.
```

#### Usage

```Python
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')  # load gpt2 tokenizer
tokenizer.pad_token = tokenizer.eos_token  # set padding token to be the same as end of sentence token
```
In the above snippet, the code is initializing a GPT-2 tokenizer with a predefined model 'gpt2'. The padding token is set to be the end of sequence token. Padding is done to ensure all sequences in a batch are of the same length, while the end of sequence token indicates the ending of a sentence.


```Python
dataset = SimpleDataset(texts, tokenizer, max_length=20)  # initialize our dataset
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)  # create a dataloader
```
The above code creates an instance of the SimpleDataset with a predefined list of texts, tokenizer, and a maximum length of 20 tokens per sequence. Then, it creates a **DataLoader** with a batch size of 2 and shuffling turned on.

This code is a typical example of how to prepare your own custom dataset for use with transformer models.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer

class SimpleDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoding = self.tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=self.max_length)
        return encoding['input_ids'].squeeze(), encoding['attention_mask'].squeeze()

# Example usage
texts = ["Hello, how are you?", "I am fine, thank you.", "What about you?"]
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
dataset = SimpleDataset(texts, tokenizer, max_length=20)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

#### NOTE: 

If you encounter this error

```FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(```
  
It is just a warning from the Transformer library (maintained by [Hugging Face](https://huggingface.co/) . The call to "resume_download" is outside this implementation. (Just make sure you have the latest versions of torch and transformer!) 

## Building the GPT Model

Next, we define the model architecture. The **GPTBlock** class represents a single transformer block, and the **SimpleGPT** class stacks multiple transformer blocks to form the complete model.

### Explanation

This snipet is creating a simplified version of the Transformer-based Generative Pre-training (GPT) model architecture. [pyTorch's neural network (torch.nn)](https://pytorch.org/tutorials/beginner/nn_tutorial.html#sphx-glr-beginner-nn-tutorial-py) library is extensively used here.

Let's break it down:

#### GPTBlock
The **GPTBlock** class encapsulates a block or a layer of the GPT architecture. Specific structures encapsulated by this class include:

1. **Layer normalization (self.ln_1 and self.ln_2)**: Layer normalization is a type of normalization technique like batch normalization. However, unlike batch normalization which normalizes across the batch dimension, layer normalization normalizes across the feature dimension. It is defined by the number of embeddings (config.n_embd). If you recall, from the [KeyTerms](#key-terms) above, an embedding is a representation of input data as vectors in a continuous vector space.

    - A **continuous vector space** is a mathematical structure where vectors can take any value within a continuum, allowing for smooth and continuous variation. This means that the components of vectors in this space can be any real number, rather than being limited to discrete values. Continuous vector spaces are used in various fields, including machine learning and natural language processing, to represent data in a way that captures semantic relationships and enables efficient computation.
 
2. **Multi-head attention (self.attn)**: This is a mechanism in Transformers that mixes individual attention mechanisms or 'heads' that jointly attend to different positions from different representational spaces. It is defined by its dimensions (config.n_embd), the number of heads (config.n_head), and the dropout rate (config.attn_pdrop).

    - In the context of neural networks, particularly in transformers, "heads" refer to the multiple attention mechanisms used in multi-head attention layers. Each head processes the input data separately and allows the model to focus on different parts of the input sequence simultaneously. This helps capture various aspects of the data, improving the model's ability to understand complex patterns.
    - Dimensions refer to the number of features or components in a vector or a tensor. In neural networks, dimensions can specify the size of input data, hidden layers, or output layers. For example, in a word embedding, dimensions indicate the length of the vector representing each word, capturing various semantic properties.
    - Dropout rate is a hyperparameter used in dropout regularization, a technique to prevent overfitting in neural networks. During training, dropout randomly sets a fraction of the neurons' outputs to zero at each iteration, specified by the dropout rate. This encourages the network to develop redundant representations, making it more robust and less likely to overfit the training data. The dropout rate is typically a value between 0 and 1, representing the probability of dropping a neuron.


3. **MLP (Multi-Layer Perceptron) (self.mlp)**: This is a **Feed-Forward Neural Network** comprised of linear layers and a GELU activation function which is a smooth approximation of the ReLU function. The output of the network is normalized and dropout is applied.

    - The **GELU (Gaussian Error Linear Unit)** function is an activation function used in neural networks. It is defined as:

        GELU
        GELU(x)=x⋅Φ(x)

        where Φ(x) is the cumulative distribution function of the standard normal distribution. Essentially, GELU smoothly blends linear and non-linear transformations of the input. It tends to perform better than the more commonly used ReLU (Rectified Linear Unit) function in many scenarios because it maintains the properties of the input's Gaussian distribution, which can lead to improved performance and training stability in deep learning models.

4. **forward** Method: This defines the forward pass for the block, applying attention, normalization and MLP sequentially. 
    
    - **Attention** is a mechanism that allows neural networks to focus on specific parts of the input data when making predictions. It dynamically weighs the importance of different input elements, enabling the model to prioritize relevant information. Attention is widely used in tasks involving sequences, such as natural language processing, where it helps models to better understand context by selectively attending to specific words or tokens in a sentence.
    
    - **Normalization** is a technique used to adjust the input data or intermediate representations in a neural network to improve training efficiency and stability. It involves scaling and shifting the data to have certain statistical properties, such as a mean of zero and a standard deviation of one. Common normalization methods include batch normalization, layer normalization, and instance normalization. These methods help to mitigate issues like internal covariate shift, accelerating convergence and improving model performance.


#### SimpleGPT
The **SimpleGPT** class is the main model class that encapsulates the final GPT architecture. Specific structures encapsulated by this class include:

- **Token Embedding (self.token_embedding)**: Embedding for the raw input tokens.
- **Position Embedding (self.position_embedding)**: Embedding for token positions in the input sequence.
- **Dropout (self.drop)**: Regularization method that drops out units in a neural network.
- **GPT Blocks (self.blocks)**: A sequence of the GPTBlock modules.
- **LayerNorm and Linear Layer**: Applied after the transformer blocks to product the final logits
    - **Logit** refers to the raw, unnormalized predictions that come from a machine learning model before they are transformed into probabilities. Specifically, in the context of classification tasks, logits are the values output by the model's final layer (before applying an activation function like softmax). These values are used to calculate probabilities that sum to one, which can then be interpreted as the likelihood of each class. The term "logit" is particularly associated with logistic regression and neural networks.
- **forward** Method: Positional embedding is added to token embedding followed by a dropout. After that for each GPT (Transformer) block, multi-head self-attention is performed followed by FeedForward Neural Network. Finally, layer normalization and a linear layer are applied.

#### Usage:
In the sample usage, the **Transformers GPTConfig** is used to set configuration variables for the GPT model such as *vocab_size, n_positions, n_embd, n_layer, n_head, resid_pdrop, embd_pdrop, and attn_pdrop*. The model is created with this configuration.

Notice how certain attributes like *n_embd* (dimensions of the GPT model embeddings) or *n_head* (numbers of heads for multi-head attention) feature in multiple places across the two classes. Token sequences are passed into the main SimpleGPT model's forward method, which handles the detailed implementation of the forward pass, taking care of processes like incorporating attention mechanisms and dropout strategies.

```Python
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
```
This line of code selects which device to use for the calculations. PyTorch, with its *torch.cuda.is_available()* function, first checks if CUDA is available. [CUDA](https://developer.nvidia.com/cuda-toolkit) is a parallel computing platform and API model created by [Nvidia](https://www.nvidia.com/) which allows software developers to use a CUDA-enabled graphics processing unit (GPU) for general purpose processing.If CUDA is available and thus a GPU is available, then "cuda" is assigned to the variable device. If there is no GPU or CUDA is not available, then "cpu" is assigned to the variable device

The last line ```model = SimpleGPT(config).to(device)```, constructs an instance of **SimpleGPT** and assigns the computations to be run on the previously selected device. 

In [ ]:
import torch.nn as nn

class GPTBlock(nn.Module):
    def __init__(self, config):
        super(GPTBlock, self).__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd)
        self.attn = nn.MultiheadAttention(config.n_embd, config.n_head, dropout=config.attn_pdrop)
        self.ln_2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.resid_pdrop)
        )

    def forward(self, x, attention_mask=None):
        attn_output, _ = self.attn(x, x, x, attn_mask=attention_mask)
        x = x + attn_output
        x = self.ln_1(x)
        mlp_output = self.mlp(x)
        x = x + mlp_output
        x = self.ln_2(x)
        return x

class SimpleGPT(nn.Module):
    def __init__(self, config):
        super(SimpleGPT, self).__init__()
        self.token_embedding = nn.Embedding(config.vocab_size, config.n_embd)
        self.position_embedding = nn.Embedding(config.n_positions, config.n_embd)
        self.drop = nn.Dropout(config.embd_pdrop)
        self.blocks = nn.ModuleList([GPTBlock(config) for _ in range(config.n_layer)])
        self.ln_f = nn.LayerNorm(config.n_embd)
        self.head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        self.config = config

    def forward(self, input_ids, attention_mask=None):
        positions = torch.arange(0, input_ids.size(1), device=input_ids.device).unsqueeze(0)
        x = self.token_embedding(input_ids) + self.position_embedding(positions)
        x = self.drop(x)

        # Adjusting attention mask for multi-head attention
        if attention_mask is not None:
            attention_mask = attention_mask.unsqueeze(1).repeat(self.config.n_head, attention_mask.size(1), 1)
            attention_mask = attention_mask.to(dtype=torch.float32)
            attention_mask = (1.0 - attention_mask) * -10000.0

        for block in self.blocks:
            x = block(x.transpose(0, 1), attention_mask)  # Transpose for multi-head attention
            x = x.transpose(0, 1)  # Transpose back to original shape

        x = self.ln_f(x)
        logits = self.head(x)
        return logits

# Example usage
from transformers import GPT2Config

config = GPT2Config(
    vocab_size=len(tokenizer),
    n_positions=20,
    n_embd=768,
    n_layer=12,
    n_head=12,
    resid_pdrop=0.1,
    embd_pdrop=0.1,
    attn_pdrop=0.1
)
device = torch.device("cpu")
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleGPT(config).to(device)

### Training the Model

The train function handles the training process, including the forward pass, loss calculation, backpropagation, and optimization.

1. **Forward Pass**:
The forward pass is the process of feeding input data through a neural network to obtain output predictions. It involves applying each layer's computations (e.g., linear transformations, activation functions) in sequence from the input layer to the output layer.

2. **Loss Calculation**:
Loss calculation involves computing a value that quantifies the difference between the model's predictions and the actual target values. This value, called the loss or cost, measures how well the model is performing. Common loss functions include mean squared error (MSE) for regression and cross-entropy loss for classification.

3. **Backpropagation**:
Backpropagation is the process of calculating the gradients of the loss function with respect to each weight in the neural network. It uses the chain rule of calculus to propagate these gradients backward through the network, from the output layer to the input layer. This information is then used to update the model's weights.

4. **Optimization**:
Optimization refers to the method of updating the model's weights to minimize the loss function. It uses the gradients calculated during backpropagation to adjust the weights in a way that reduces the loss. Common optimization algorithms include stochastic gradient descent (SGD), Adam, and RMSprop.

5. **Epoch**:
epoch is one complete pass through the entire training dataset. During an epoch, the model processes each example in the training set once, which involves making predictions (forward pass), calculating the loss, performing backpropagation to compute gradients, and updating the model's weights. Multiple epochs are often required to train a model effectively, as they allow the model to learn and improve its performance through iterative weight updates.

6. **Stochastic Gradient Descent (SGD)**:
SGD is an optimization technique used in machine learning and deep learning to minimize the loss function. Unlike traditional gradient descent, which uses the entire dataset to compute gradients and update weights, SGD updates weights using a single randomly selected data point (or a small batch of data points) at each iteration. This approach introduces more noise into the training process, which can help the model escape local minima and converge faster. However, it may require more iterations to reach the optimal solution due to the noisy updates.

7. **Softmax**: 
Softmax is an activation function often used in the output layer of a neural network for classification tasks. It converts raw logits (real-valued scores) into probabilities by exponentiating each logit and normalizing by the sum of the exponentiated logits. This ensures that the output probabilities sum to 1, making it suitable for multi-class classification problems. 

#### Explanation

- **train Function**: Handles the training loop for the model.
    - **model.train()**: Sets the model to training mode.
    - **Epoch Loop**: Iterates over the number of epochs.
    - **DataLoader Loop**: Iterates over batches of data from the DataLoader.
    - **Forward Pass**: Passes the input IDs and attention mask through the model to get the output logits.
    - **Loss Calculation**: Shifts the logits and labels to align predictions and targets, then calculates the loss using cross-entropy.
    - **Backward Pass and Optimization**: Performs backpropagation and updates the model parameters using the optimizer.
    - **Loss Tracking**: Accumulates the total loss for each epoch and prints the average loss at the end of each epoch.

#### Usage


1. **optimizer = optim.Adam(model.parameters(), lr=1e-4)**:
This line of code is creating an Adam optimizer. The Adam optimizer is a type of **stochastic gradient descent** method commonly used for training deep learning models. The first argument is the parameters of your model, while the lr parameter sets the learning rate, which determines how large of a step the optimizer takes during gradient descent.

2. **criterion = nn.CrossEntropyLoss()**:
*nn.CrossEntropyLoss()* is a loss function that is often used in multiclass classification problems. It applies a softmax function to the output of the model, and then computes the negative log-likelihood loss between the prediction and the ground truth. This value is minimized during the training process.


In [ ]:
import torch.optim as optim

def train(model, dataloader, optimizer, criterion, epochs=5, device='cuda'):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for input_ids, attention_mask in dataloader:
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            shift_logits = outputs[..., :-1, :].contiguous()
            shift_labels = input_ids[..., 1:].contiguous()
            loss = criterion(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(dataloader)}")

# Example usage
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
train(model, dataloader, optimizer, criterion, epochs=5, device=device)

### Generating Text

The generate_text function allows you to generate text from a trained model given a prompt.

#### Explanation

- **generate_text** Function: Generates text from a trained GPT model.
    - **model.eval()**: Sets the model to evaluation mode.
    - **Tokenization**: Encodes the prompt into input IDs using the tokenizer.
    - **Generation Loop**: Iterates up to max_length to generate tokens sequentially.
    - **Model Output**: Passes the generated sequence through the model to get logits for the next token.
    - **Next Token Prediction**: Selects the token with the highest probability from the logits.
    - **Sequence Update**: Appends the predicted token to the generated sequence.
    - **Stopping Condition**: Breaks the loop if the end-of-sequence token is generated.
    - **Decoding**: Converts the generated sequence of tokens back to text using the tokenizer.

In [ ]:
def generate_text(model, tokenizer, prompt, max_length=50, device='cuda'):
    model.eval()
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    generated = input_ids

    for _ in range(max_length):
        outputs = model(generated)
        next_token_logits = outputs[:, -1, :]
        next_token = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
        generated = torch.cat((generated, next_token), dim=1)
        if next_token.item() == tokenizer.eos_token_id:
            break

    generated_text = tokenizer.decode(generated[0], skip_special_tokens=True)
    return generated_text

# Example usage
prompt = "Once upon a time"
generated_text = generate_text(model, tokenizer, prompt, device=device)
print(generated_text)

#### NOTE: 

If you see RuntimeErrors (specifically CUDA errors)

Change these lines of code above
    ```Python
    #device = torch.device("cpu")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ```

to look like this
        ```Python
    device = torch.device("cpu")
    #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    ```

### Conclusion
This guide provided a step-by-step explanation of implementing a simple GPT model using PyTorch. We covered creating a custom dataset, building the GPT model, training the model, and generating text. This implementation demonstrates the fundamental concepts behind the GPT architecture and can be extended for more complex applications.

By following this guide, you should now have a basic understanding of how to create, train, and use a simple GPT model. You can further experiment with different configurations, larger datasets, and additional techniques to improve the model's performance and capabilities.